<a href="https://colab.research.google.com/github/nonoumasy/twitter_sent_analysis/blob/master/twitter_sent_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 3.5MB/s 


In [0]:
import tweepy
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [0]:
#consumer_key = '4V4xcJCBiQhd88p9yudwerbNN' 
consumer_secret = '6XRUHDS8Fm2sZrX8uyH6nLI2ApiWRJCJhGXOitbJqEfUYmDZks'
access_token = '2942077020-NaRtgEdNlpGQwSKIckxdO8O5CCdHXASgD8mgUk2'
access_token_secret = 'Wt8ZPkNR0Ax3Pv334LTS3Pd6OVQFvph80KgMI6JXyGkES'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [0]:
def get_related_tweets(key_word):
  twitter_users = []
  tweet_time = []
  tweet_string = [] 
  for tweet in tweepy.Cursor(api.search,q=key_word, count=1000).items(1000):
          if (not tweet.retweeted) and ('RT @' not in tweet.text):
              if tweet.lang == "en":
                  twitter_users.append(tweet.user.name)
                  tweet_time.append(tweet.created_at)
                  tweet_string.append(tweet.text)
                  print([tweet.user.name,tweet.created_at,tweet.text])
  df = pd.DataFrame({'name':twitter_users, 'time': tweet_time, 'tweet': tweet_string})
  
  return df

In [44]:
finance_df = get_related_tweets('stock market')

['Burn Unit', datetime.datetime(2020, 1, 28, 19, 8, 4), '@IvankaTrump Peace and prosperity by deciding America and artificially pumping the stock market. Perfect for a civi… https://t.co/CfPyGQZMiR']
['Simon Hobbs', datetime.datetime(2020, 1, 28, 19, 8, 1), 'Beware #hotelowners margin-pressure (c.f. brands) o/next 3 years. Economic rebound likely slower than stock market… https://t.co/SzmThCZ31e']
['LogicalJR', datetime.datetime(2020, 1, 28, 19, 6, 52), '@darinp2 @Black_DJTrump @charliekirk11 Trump:\nLowest unemployment rate in more than 50 years\nAverage GDP higher tha… https://t.co/NALnYBa6hk']
['TravelerA5', datetime.datetime(2020, 1, 28, 19, 6, 39), 'Has ANYONE really thought about life after\ndemocrats taking over the WH, House, Senate ?\n\nWhat that would mean to o… https://t.co/SMbluCSfC7']
['AIM Researcher', datetime.datetime(2020, 1, 28, 19, 6, 30), 'Sad to see this RNS arrive for #MPAY. https://t.co/PbkjA7apnm\n\nA good company, used to have a tremendous asset, bei… https://t

In [45]:
finance_df.head()

,name,time,tweet
0,Burn Unit,2020-01-28 19:08:04,@IvankaTrump Peace and prosperity by deciding ...
1,Simon Hobbs,2020-01-28 19:08:01,Beware #hotelowners margin-pressure (c.f. bran...
2,LogicalJR,2020-01-28 19:06:52,@darinp2 @Black_DJTrump @charliekirk11 Trump:\...
3,TravelerA5,2020-01-28 19:06:39,Has ANYONE really thought about life after\nde...
4,AIM Researcher,2020-01-28 19:06:30,Sad to see this RNS arrive for #MPAY. https://...


In [0]:
#vaderSentiment
analyzer = SentimentIntensityAnalyzer()

finance_df['vader_sentiment'] = finance_df['tweet'].apply(lambda x : analyzer.polarity_scores(x))

#using TextBlob
finance_df['tb_sentiment'] = finance_df['tweet'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)

In [52]:
finance_df_pos = finance_df[finance_df['tb_sentiment'] > 0.0]
finance_df_neg = finance_df[finance_df['tb_sentiment'] < 0.0]
print("Number of Positive Tweets", len(finance_df_pos))
print("Number of Negative Tweets", len(finance_df_neg))

Number of Positive Tweets 173
Number of Negative Tweets 68


In [54]:
finance_df[finance_df['tb_sentiment'] < 0.0]

,name,time,tweet,vader_sentiment,tb_sentiment
3,TravelerA5,2020-01-28 19:06:39,Has ANYONE really thought about life after\nde...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",-0.056250
10,Mielo,2020-01-28 19:05:36,@motokogames @KozoroMusic it affected the stoc...,"{'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'comp...",-0.155556
11,FinanzLinksUSA,2020-01-28 19:05:04,Stock Market Update: Gold Comes A Bit Off Mult...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",-0.062500
15,UC Berkeley,2020-01-28 19:05:01,What’s behind the stock market’s gains over th...,"{'neg': 0.106, 'neu': 0.777, 'pos': 0.117, 'co...",-0.250000
38,motoko💀🥀🖤,2020-01-28 19:00:52,"@KozoroMusic Sharing false symptom reports, fa...","{'neg': 0.248, 'neu': 0.64, 'pos': 0.112, 'com...",-0.433333
...,...,...,...,...,...
409,Joseph Markowitz,2020-01-28 17:52:12,@ClodaghSnarks I’m terribly ashamed of 3.5 % u...,"{'neg': 0.484, 'neu': 0.516, 'pos': 0.0, 'comp...",-0.583333
411,Matt Darling 🌐🌅,2020-01-28 17:51:44,@AlanMCole @economeager I'm not getting really...,"{'neg': 0.257, 'neu': 0.743, 'pos': 0.0, 'comp...",-0.308333
415,NSawatzky,2020-01-28 17:51:02,"@thenelsonsmith Yeah, plenty of 'average' folk...","{'neg': 0.15, 'neu': 0.762, 'pos': 0.088, 'com...",-0.150000
423,❌Patriots🇺🇸Pulse❌,2020-01-28 17:49:27,@NunesCellmate @TBeto777 You are in a great mi...,"{'neg': 0.151, 'neu': 0.699, 'pos': 0.151, 'co...",-0.100000
